# Progetto Finale del Corso "Big Data"- Parte I: Analisi di Wikipedia

##  EDA
E’ necessario svolgere un’attività di EDA per analizzare e valutare statisticamente tutto il contenuto informativo offerto da Wikipedia. Il bump che ti viene fornito, possiede le seguenti categorie: 

* 'culture',
* 'economics',
* 'energy',
* 'engineering',
* 'finance',
* 'humanities',
* 'medicine',
* 'pets',
* 'politics',
* 'research',
* 'science',
* 'sports',
* 'technology',
* 'trade',
* 'transport'


Nello specifico, quello che ti viene richiesto consiste nel calcolare, per ogni categoria, le seguenti informazioni.

* Numero di articoli
* Numero medio di parole utilizzate
* Numero massimo di parole presenti nell’articolo più lungo
* Numero minimo di parole presenti nell’articolo più corto
* Per ogni categoria, individuare la nuvola di parole più rappresentativa



## Svolgimento del progetto Parte-I : 

La prima parte del progetto consiste nel calcolare per ogni categoria alcune informazioni. Iniziamo subito caricando il dataset su cui dovremo lavorare.

Nota. Eliminiamo la tabella in caso esista già per evitare errori

In [0]:
%sql
DROP TABLE IF EXISTS wikipediaDB

In [0]:
#!wget https://proai-datasets.s3.eu-west-3.amazonaws.com/wikipedia.csv
import pandas as pd
dataset = pd.read_csv('/databricks/driver/wikipedia.csv')
spark_df = spark.createDataFrame(dataset)
spark_df = spark_df.drop("Unnamed: 0")
spark_df.write.mode("overwrite").saveAsTable("wikipediaDB")
display(spark_df)

Osserviamo il dataset. Contiene 4 colonne :
* title: indica il titolo dell'articolo
* summary: contiene l'introduzione dell'articolo
* documents: contiene l'articolo completo
* categoria: contiene la categoria associata all'articolo


### Numero di articoli per categoria

Per ricavare l'informazione per ogni categoria, utilizziamo una query SQL. Sfruttiamo la funzione 'group by' sulla colonna categoria.

In [0]:
wiki_1 = spark.sql("""
    select categoria, count(documents) as N_articoli
    from wikipediaDB
    group by categoria
""")
display(wiki_1)

## Numero medio di parole per gli articoli per ogni categoria


Il secondo task consiste nel calcolare il numero medio di parole degli articoli per ogni categoria. Per prima cosa dobbiamo splittare ogni articolo nelle sue parole e calcolarne la lunghezza.

In [0]:
wiki_temp = spark.sql("""
    select categoria, SPLIT(documents, " ") AS Splitted_words
    from wikipediadb
""")
wiki_temp.createOrReplaceTempView("temp_table")
display(wiki_temp)

La funzione di split ha funzionato correttamente ma sono presenti alcuni caratteri speciali o segni di punteggiatura. 

In [0]:
wiki_temp = spark.sql("""
    select categoria, FILTER(SPLIT(documents, " "), word -> NOT regexp_like(word, '^[!"#$%&)(*,-./:;<=>?@–\\^_`{|}~+]+$'))  AS Splitted_words
    from wikipediadb
""")
wiki_temp.createOrReplaceTempView("temp_table")
display(wiki_temp)

Usiamo ora le funzioni AVG e Length per calcolare la media e la lunghezza degli articoli per ogni categoria

In [0]:
wiki_2 = spark.sql("""
    select categoria, INT(AVG(SIZE(Splitted_words))) as avg_num_words
    from temp_table
    group by categoria
""")
display(wiki_2)

## Numero massimo di parole presenti nell’articolo più lungo

Per trovare l'articolo più lungo, sfruttiamo sempre la view che abbiamo creato e utilizziamo la funzione max su lenght

In [0]:
wiki_3 = spark.sql("""
    select categoria, MAX(size(Splitted_words)) as Lunghezza_Max
    from temp_table
    group by categoria
""")
display(wiki_3)

## Numero minimo di parole presenti nell’articolo più lungo

In questo caso, cerchiamo l'articolo più corto. Usiamo la funzione Min


In [0]:
wiki_4 = spark.sql("""
    select categoria, MIN(SIZE(Splitted_words)) as Lunghezza_Min
    from words_array
    group by categoria
""")
display(wiki_4)

## Nuvola di Parole

Per creare la wordcloud per ogni categoria, dobbiamo preprocessare il testo. Iniziamo importando le librerie necessarie

In [0]:
# Se richiesto installare le librerie
!pip install wordcloud
!pip install nltk

import wordcloud
import nltk
from nltk.corpus import stopwords

# Scarichiamo le stopword che passeremo alla funzione wordcloud 
nltk.download('stopwords')
print(stopwords.words('english'))
stop_words = set(stopwords.words('english'))

In [0]:
# source https://stackoverflow.com/questions/49468362/combine-text-from-multiple-rows-in-pyspark
from pyspark.sql.functions import collect_list
from pyspark.sql.functions import concat_ws 
grouped_df = spark_df.groupby('categoria').agg(collect_list('documents').alias("documents"))
grouped_df.withColumn("documents", concat_ws(" ", "documents")).show()

In [0]:
import matplotlib.pyplot as plt

for category in categories:
    text = grouped_df.select("merged_documents").where(grouped_df.category==categoria).head()[0]
    wordcloud = WordCloud(width=2000, height=2000,
                          background_color ='black', margin=1,
                          stopwords = stop_words,
                          ).generate(text)
    plt.subplot(3,5)
    plt.axis("off")
    plt.title("Categoria: "+categoria)
    plt.tight_layout(pad=2)
    plt.imshow(wordcloud,interpolation='bilinear')

plt.show()

### Ripetiamo ora tutte le funzioni per colonna "summary".

In [0]:
wiki_temp_bis = spark.sql("""
    select categoria, FILTER(SPLIT(summary, " "), word -> NOT regexp_like(word, '^[!"#$%&)(*,-./:;<=>?@–\\^_`{|}~+]+$'))  AS Splitted_words
    from wikipediadb
""")
wiki_temp.createOrReplaceTempView("temp_table_bis")
display(wiki_temp_bis)

In [0]:
wiki_2_bis = spark.sql("""
    select categoria, INT(AVG(SIZE(Splitted_words))) as avg_num_words
    from temp_table
    group by categoria
""")
display(wiki_2_bis)

In [0]:
wiki_3_bis = spark.sql("""
    select categoria, MAX(SIZE(Splitted_word)) as Lunghezza_Max
    from temp_table
    group by categoria
""")
display(wiki_3_bis)

In [0]:
wiki_4_bis = spark.sql("""
    select categoria, MIN(SIZE(Splitted_words)) as Lunghezza_Min
    from words_array
    group by categoria
""")
display(wiki_4_bis)

In [0]:
grouped_df_bis = spark_df.groupby('categoria').agg(collect_list('summary').alias("summary"))
grouped_df_bis.withColumn("summary", concat_ws(" ", "summary")).show()

In [0]:
for categoria in categories:
    text = grouped_df_bis.select("merged_documents").where(grouped_df_bis.category==categoria).head()[0]
    wordcloud = WordCloud(width=2000, height=2000,
                          background_color ='black', margin=1,
                          stopwords = stop_words,
                          ).generate(text)
    plt.subplot(3,5)
    plt.axis("off")
    plt.title("Categoria: "+categoria)
    plt.tight_layout(pad=2)
    plt.imshow(wordcloud,interpolation='bilinear')

plt.show()